# Test AES Design
## This Jupyter notebook will run the design on FPGA 
### WHen configured can also display the maximum frequency of the design

In [ ]:
from pynq import Overlay, Clocks

In [ ]:
# Function to write instructions to registers
def write(idata):
    f = open(idata, "r")
    for count, line in enumerate(f.readlines()):
        cpu_ip.write(int(count*4),int(line, 16))
    

# Function to read RISCV 32 Registers
def read():
    out = []
    for i in range(32,64):
        out.append(cpu_ip.read(int(i*4)))
    return out


In [ ]:
clocks = [50, 100, 125, 142, 200]
for i in range(0,6):
    print("Testing "+"idata"+str(i+1)+".txt")
    f = open("expout"+str(i+1)+".txt", "r")        # Expected output file
    out_crct = []
    for line in f.readlines():
        out_crct.append(int(line))
    for clock in clocks:
        ol_cpu = Overlay('overlay/harness_axi.bit')    # Programming the bitstream onto the FPGA
        cpu_ip = ol_cpu.harness_axi_ip_v1_0_0          # RISCV CPU Verilog IP
        Clocks.fclk1_mhz = clock                       # Set clock frequency in MHz
        cpu_ip.write(int(64*4),1)                      # Assign reset to 1
        write("idata"+str(i+1)+".txt")                 # Write instructions
        cpu_ip.write(int(64*4),1)                      # Assign reset to 1
        cpu_ip.write(int(64*4),0)                      # Assign reset to 0
        out = read()                                   # Read registers
        
        
        # Increase the clock frequency untill timing violations occur
        if out==out_crct:                              # Check if expected and obtained register values
            print("\t", "Frequency at", clock, "MHz: Passed")
            clock += 50
        else:
            print("\t", "Frequency at", clock, "MHz: Not passed")
#             Use below lines for debug
#             print("Expected result: ", out_crct)
#             print("Obtained result: ", out)


KEY 128'h00010203 04050607 08090a0b 0c0d0e0f
IN  128'hB9D1C48E 348FE771 FA464A77 A178FB07
OUT 128'h95F88473 69A8573D 76AF987A B30A5DE2

In [ ]:
ol_aes = Overlay('overlay/harness_axi.bit')    # Programming the bitstream onto the FPGA
aes_ip = ol_aes.harness_axi_ip_v1_0_0          # RISCV CPU Verilog IP
Clocks.fclk1_mhz = clock                       # Set clock frequency in MHz
aes_ip.write(int(64*4),0)                      # Assign reset to 0, we have a active low reset
aes_ip.write(int(0*4),0xa178fb07)              #in0
aes_ip.write(int(1*4),0xfa464a77)              #in1
aes_ip.write(int(2*4),0x348fe771)              #in2
aes_ip.write(int(3*4),0xb9d1c48e)              #in3

aes_ip.write(int(4*4),0x0c0d0e0f)              #key0
aes_ip.write(int(5*4),0x08090a0b)              #key1
aes_ip.write(int(6*4),0x04050607)              #key2
aes_ip.write(int(7*4),0x00010203)              #key3

aes_ip.write(int(8*4),0x1) #KRdy
aes_ip.write(int(9*4),0x1) #DRdy
aes_ip.write(int(64*4),1)                      # Assign reset to 1 , remove the reset

out0 = 0x0
out1= 0x0
out2= 0x0
out3= 0x0
while 1:
    if aes_ip.read(int(36*4) == 1):
        continue
    else:
        out0 = aes_ip.read(int(32*4))
        out1 = aes_ip.read(int(33*4))
        out2 = aes_ip.read(int(34*4))
        out3 = aes_ip.read(int(35*4))

print("out0: ", out0)
print("out1: ", out1)
print("out2: ", out2)
print("out3: ", out3)



